# <center> Car Price Prediction Model Building

<img src="Red-sport-car-isolated-on-transparent-background-PNG.png"     img>

In [ ]:
!pip3 install bayesian-optimization

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings


from bayes_opt import BayesianOptimization

import sklearn 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,PolynomialFeatures,StandardScaler


from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,\
VotingRegressor,ExtraTreesRegressor,HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import svm
from xgboost import XGBRegressor


from sklearn.metrics import r2_score, mean_squared_error as mse, mean_absolute_error as mae
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix


from datetime import datetime

warnings.filterwarnings("ignore")

# Load Data

In [2]:
df=pd.read_csv("cars_data_after_feature_engenniring.csv")
df.drop(columns=["Unnamed: 0"],inplace=True)
#df=df[["Kilométrage","Puissance_fiscale","Age","N_features","Price"]]
df.head()

,Power,Kilométrage,Boite_de_vitesses,Puissance_fiscale,Première_main,Carrosserie,Price,Mark,Model,Age,N_features
0,Diesel,300000.0,Manuelle,7.0,0,Citadine,45000.0,PEUGEOT,206,21,5
1,Diesel,70000.0,Manuelle,6.0,1,Citadine,158000.0,NISSAN,Micra,2,31
2,Diesel,155000.0,Automatique,9.0,1,Suv et 4x4,225000.0,HYUNDAI,Santa fe,7,38
3,Diesel,154000.0,Manuelle,11.0,1,Utilitaire (van),300000.0,FIAT,Ducato,17,0
4,Diesel,93000.0,Manuelle,9.0,1,Utilitaire (van),162000.0,PEUGEOT,Boxer,7,16


# Data Preprocessing 

## 1- OneHotEncoding

In [3]:
onehot_df=pd.get_dummies(df)
onehot_df

,Kilométrage,Puissance_fiscale,Première_main,Price,Age,N_features,Power_Diesel,Power_Electrique,Power_Essence,Power_Hybride,...,Model_Xlv,Model_Xsara,Model_Xsara picasso,Model_Yaris,Model_Yaris cross,Model_Yeti,Model_Ypsilon,Model_Yrv,Model_Zafira,Model_Zx
0,300000.0,7.0,0,45000.0,21,5,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,70000.0,6.0,1,158000.0,2,31,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,155000.0,9.0,1,225000.0,7,38,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,154000.0,11.0,1,300000.0,17,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,93000.0,9.0,1,162000.0,7,16,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19091,163000.0,6.0,0,58000.0,17,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19092,114000.0,9.0,1,180000.0,8,21,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19093,140000.0,8.0,1,145000.0,7,14,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19094,290000.0,12.0,1,180000.0,14,33,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
scaler=StandardScaler()
scaled_kilom=scaler.fit_transform(df[["Kilométrage"]])
scaled_kilom

array([[ 1.82887466],
       [-1.0246454 ],
       [ 0.02991636],
       ...,
       [-0.15618277],
       [ 1.70480857],
       [-0.29265547]])

In [26]:
# define predictor(X) and target(Y)
X=onehot_df.drop(columns="Price")

Scaled_X=StandardScaler().fit_transform(X)
Y=onehot_df["Price"]

Scaled_X

array([[ 1.82887466, -0.20807683, -0.78336171, ..., -0.01772853,
        -0.02895815, -0.00723669],
       [-1.0246454 , -0.81993936,  1.2765495 , ..., -0.01772853,
        -0.02895815, -0.00723669],
       [ 0.02991636,  1.01564821,  1.2765495 , ..., -0.01772853,
        -0.02895815, -0.00723669],
       ...,
       [-0.15618277,  0.40378569,  1.2765495 , ..., -0.01772853,
        -0.02895815, -0.00723669],
       [ 1.70480857,  2.85123579,  1.2765495 , ..., -0.01772853,
        -0.02895815, -0.00723669],
       [-0.29265547,  0.40378569, -0.78336171, ..., -0.01772853,
        -0.02895815, -0.00723669]])

In [59]:
# define train and test data 
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=123)
print(x_train.shape,
      y_train.shape,
      x_test.shape,
      y_test.shape)

(13367, 606) (13367,) (5729, 606) (5729,)


In [61]:
model_=[]
score=[]
def predict(ml_model):
    print('Model is: {}'.format(ml_model))
    model= ml_model.fit(x_train,y_train)
    print("Training score: {}".format(model.score(x_train,y_train)))
    predictions = model.predict(x_test)
    print("Predictions are: {}".format(predictions[0:5]))
    print('\n')
    r2score=r2_score(y_test,predictions) 
    print("r2 score is: {}".format(r2score))
          
    print('MAE:{}'.format(mse(y_test,predictions)))
    print('MSE:{}'.format(mse(y_test,predictions)))
    print('RMSE:{}'.format(np.sqrt(mse(y_test,predictions))))
    model_.append(type(ml_model).__name__)
    score.append(np.round(r2score,2))
    
     
    sns.distplot(y_test-predictions)   

### Linear Regression

In [ ]:
lr=LinearRegression()
predict(lr)

In [ ]:
lor=LogisticRegression()#,Ridge,Lasso
predict(lor)

In [ ]:
rr=Ridge()
predict(rr)

In [ ]:
las=Lasso()
predict(las)

In [ ]:
dtr=DecisionTreeRegressor()
predict(dtr)

In [ ]:
rfr=RandomForestRegressor()
predict(rfr)

In [ ]:
svr=svm.SVR()
predict(svr)

In [ ]:
gbr=GradientBoostingRegressor()
predict(gbr)

In [ ]:
abr=AdaBoostRegressor()
predict(abr)

In [ ]:
hgbr=HistGradientBoostingRegressor()
predict(hgbr)

In [ ]:
knnr=KNeighborsRegressor()
predict(knnr)

In [ ]:
mlpr=MLPRegressor()
predict(mlpr)

In [ ]:
xgbr=XGBRegressor(tree_method="hist")
predict(xgbr)

# Cross Validation

In [ ]:
def cross_val(model,folds):
    print(" the model is :",type(model).__name__)
    scores=cross_val_score(estimator=model,X=X,y=Y,cv=folds,scoring="r2",n_jobs=-1)
    print("The validation score is",scores.mean())
    predictions=cross_val_predict(estimator=model,X=X,y=Y,cv=folds,n_jobs=-1)
    ax=sns.distplot(Y-predictions)
    return ax

In [ ]:
cross_val(xgbr,5)

In [ ]:
cross_val(gbr,4)

In [ ]:
cross_val(hgbr,4)

In [ ]:
cross_val(rfr,4)

# Hyper Tunning 

#### Manual hypertunning:

In [ ]:
xgb_param=XGBRegressor(
                        colsample_bytree=0.932,
                        colsample_bylevel=0.932,
                        colsample_bynode=0.932,
                        subsample=1,
                        tree_method="hist",
                        #max_delta_step=0,
                        max_depth=4,
                        learning_rate=0.40,
                        reg_alpha=0.95,
                        reg_lambda=0.95,
                        n_estimators=600,#,800,900,1000],#,800,900,1000],
                        min_child_weight=3,
                        random_state=123
                        )
predict(xgb_param)

#### RandomizedSearchCV

In [ ]:
def hyper_tunning(model,params):
    rscv=RandomizedSearchCV(estimator=model,
                           param_distributions=params,
                          n_iter=200,cv=3,
                          random_state=123,
                          n_jobs=-1)
    prediction=predict(rscv)
    return prediction,"best parameters:",rscv.best_params_




### Hist Gradient Boosting Regressor

In [ ]:

param={"learning_rate":[0,0.1,0.2,0.3,0.4,0.6,0.5,0.7,0.8,0.9,1],
       "loss":['squared_error', 'absolute_error', 'poisson', 'quantile'],
      "l2_regularization":[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
      "max_depth":[1,2,3,4,5,6,7,8,9,10],
       "max_leaf_nodes":[25,26,27,28,29,30,31,32,33,34,35],
       "max_iter":[80,90,100,110,120],
       "max_bins":[250,255,260],
       "n_iter_no_change":[10,20,30]
       
       
      }
hgbr=HistGradientBoostingRegressor(random_state=123)
hyper_tunning(hgbr,param)

## X Gradient Boosting Regressor

In [63]:
val=np.arange(0,6,0.99)
val

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [79]:
xgbr_param={
       #"booster":["gbtree", "gblinear","dart"],
       #"objective":["reg:squarederror","reg:squaredlogerror","reg:absoluteerror"],
       "colsample_bytree":val,
       "colsample_bylevel":val,
       "colsample_bynode":val,
       "reg_alpha":val,
       "reg_lambda":val,
       "subsample":val,
       "gamma":[0,1,2,3],#,4,5],
       "learning_rate":[0.3,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.4,0.42,0.45],#to 0.3
       #"max_delta_step":[1,2,3,4],#5,6,7],
       "max_depth":[1,2,3,4,5,6],#5,6,7,8],
       #"max_bin":[10,20,30],
       "n_estimators":[500,520,550,570,590,600,700,800],#900,1000],#,800,900,1000],
       #"min_child_weight":[1,2,3,4,5],
       #"max_cat_threshold":[1,5,10]
        }
#xgbr=XGBRegressor(tree_method="hist")

In [ ]:
hyper_tunning(xgbr,param)

### Random Forest Regressor

In [ ]:
rfr=RandomForestRegressor()
param={"n_estimators":[100,200,300,400],
    "criterion":['squared_error',"absolute_error", "friedman_mse", "poisson"],
    "max_depth":[1,2,3,4,5],
    "min_samples_split":[2,3,4,5,6],
    "min_samples_leaf":[1,2,3,4,5],
    "min_weight_fraction_leaf":[0.0,0.1,0.2,0.3],
    "max_features":[1.0,2,3,4,5,6],
    "max_leaf_nodes":[0,1,2,3],
    "ccp_alpha":[0.0,0.2,0.1,0.3,0.4],
    "max_samples":[0,1,2,3,4]}
#hyper_tunning(rfr,param)

## Hyper tunning using Bayesian Optmization

In [ ]:
params={"n_estimators":(100,600),
       #"ccp_alpha":[0,0.9],
        #"max_depth":(1,6)
       }
params

In [ ]:
def objective(n_estimators):
  # Train a model with the given hyperparameters.
  rfr=RandomForestRegressor(n_estimators=int(n_estimators))
  rfr.fit(x_train, y_train)

  # Evaluate the model on the test set.
  predictions=rfr.predict(x_test)
  error = r2_score(predictions,y_test)

  return error

optimizer = BayesianOptimization(f=objective, pbounds=params,
                                random_state=123)

optimizer.maximize(n_iter=10)

In [ ]:
optimizer.max["params"]

In [ ]:
optimizer.max["target"]

### Using GridSearchCV

In [77]:
def grid_search(model,params):
    rs_cv=GridSearchCV(model,params,cv=4,
                       scoring="neg_mean_squared_error")
    
    return predict(rs_cv),print("best parameters are:",rs_cv.best_params_)


In [80]:
grid_search(XGBRegressor(),xgbr_param)

Model is: GridSearchCV(cv=4,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                         'learning_rate': [0.3, 0.31, 0.32, 0.33, 0.34, 0.35,
                                           0.36, 0.37, 0.38, 0.39, 0.4, 0.42,
                                           0.45],
                         'max_depth': [1, 2, 3, 4, 5, 6],
                  

MemoryError: 

NameError: name 'rs_cv' is not defined

In [ ]:
all_columns=set(onehot_df.columns)

In [ ]:
my_car=pd.DataFrame({'Power':"Diesel", 'Kilométrage':120000, 'Boite_de_vitesses':"Manuelle", 'Puissance_fiscale':6,
       'Première_main':0, 'Carrosserie':"Hatchback", 'Mark':"FORD",'Model':'Focus', 'Age':7,
       'N_features':12},index=[0])
my_car.index

In [ ]:
def add_col(test_data,trained_data=onehot_df):
    all_columns=set(trained_data.columns)
    test_data_dummies=pd.get_dummies(test_data)
    test_columns=test_data_dummies.columns
    diff_columns=all_columns.difference(test_columns)
    needed_col=pd.DataFrame(dict(zip(diff_columns,np.zeros(len(diff_columns)))),
                            index=test_data.index)
    final_df=pd.concat([test_data,needed_col],axis=1)
    #final_df=final_df[trained_data.columns]
    final_df.drop(columns="Price",inplace=True)
    return final_df#final_df


In [ ]:
test_df=add_col(my_car)
set(test_df.columns).difference(set(onehot_df.columns))


In [ ]:
(156137-110000)/110000*100

In [ ]:
# the best 
xgbr.predict(add_col(my_car))

In [ ]:
(120000-110000)/110000*100

In [ ]:
gbr.predict(final_data)

In [ ]:
hgbr.predict(final_data)

In [ ]:
rfr.predict(final_data)

In [ ]:
rr.predict(final_data)

In [ ]:
las.predict(final_data)

In [ ]:
dtr.predict(final_data)

In [ ]:
df.columns

In [ ]:
test_data=pd.read_csv("cleaned_car_test_data.csv")

features=['Dvd_cd_mp3', 'Jantes_aluminium', 'Alarme', 'Airbags',
       'Aide_parking', 'Climatisation_auto', 'Abs',
       'Système_de_navigation_gps', "Carnet_d'entretien", 'Vitres_éléctriques',
       'Anti_patinage', 'Esp', 'Capote_électrique', 'Projecteurs_xenon',
       'Rétroviseur_extérieur_électrique', 'Toit_ouvrant_panoramique',
       'Toit_ouvrant', 'Intérieur_cuir', 'Siège_chauffant', 'Siége_electrique',
       'Siége_sport', 'Vitres_surteintées', 'Volant_reglable',
       'Anti_démarrage', 'Volant_sport', 'Contrôle_de_pression_des_pneus',
       'Détecteur_de_pluie', 'Climatisation_multizone', 'Direction_assistée',
       'Keyless_go', 'Ordinateur_de_bord', 'Radio_commande_au_volant',
       'Régulateur_de_vitesse', 'Anti_brouillard', 'Affichage_tête_haute',
       'Suspension_sport', 'Frein_de_parking_automatique', 'Non_fumeur',
       "Système_d'identification_du_conducteur", 'Radar_de_recul',
       'Limiteur_de_vitesse', 'Verrouillage_centralisé_à_distance']

test_data['N_features']=test_data[features].sum(axis=1)


In [ ]:
test_data["Age"]=datetime.now().year-test_data["Year"]
test_data

In [ ]:
encoded_test_data=pd.get_dummies(test_data)
#encoded_test_data=pd.get_dummies(test_data)


In [ ]:
X=encoded_test_data.drop(columns="Price")
y=encoded_test_data["Price"]